In [1]:
import requests
import pandas as pd
import numpy as np
from datetime import date
import import_ipynb
import covid_dict as cd

importing Jupyter notebook from covid_dict.ipynb


1) [Remover colunas dos atributos desconsiderados](#remove_column)  
2) [Arrumando os dados](#data_fix)  
3) [Gerando a matrix binária](#binary_matrix)  
4) [PaNDa](#panda)

[Dicionário de dados](covid_dict.ipynb)

In [ ]:
# csv_url = "https://ti.saude.rs.gov.br/covid19/download"
# today = date.today()
# dataset = f'dataset-{today}.csv'

# req = requests.get(csv_url)
# url_content = req.content
# csv_file = open(dataset, 'wb')

# csv_file.write(url_content)
# csv_file.close()

In [2]:
df = pd.read_csv('amostra.csv', sep=';')
df.tail()

,COD_IBGE,MUNICIPIO,COD_REGIAO_COVID,REGIAO_COVID,SEXO,FAIXAETARIA,CRITERIO,DATA_CONFIRMACAO,DATA_SINTOMAS,DATA_EVOLUCAO,...,DATA_INCLUSAO_OBITO,DATA_EVOLUCAO_ESTIMADA,RACA_COR,ETNIA_INDIGENA,PROFISSIONAL_SAUDE,BAIRRO,HOSPITALIZACAO_SRAG,FONTE_INFORMACAO,PAIS_NASCIMENTO,PES_PRIV_LIBERDADE
166,430010,AGUDO,1,SANTA MARIA - R01 R02,Masculino,40 a 49,RT-PCR,14/09/2020,07/09/2020,NaN,...,NaN,21/09/2020,BRANCA,NAO ENCONTRADO,NAO,CENTRO,NAO,E-SUS,BRASIL,NAO
167,430010,AGUDO,1,SANTA MARIA - R01 R02,Feminino,50 a 59,TESTE RÁPIDO,25/08/2020,10/08/2020,NaN,...,NaN,24/08/2020,BRANCA,NAO ENCONTRADO,NAO,CENTRO,NAO,E-SUS,BRASIL,NAO
168,430010,AGUDO,1,SANTA MARIA - R01 R02,Feminino,20 a 29,TESTE RÁPIDO,21/09/2020,17/09/2020,NaN,...,NaN,01/10/2020,NAO INFORMADO,NAO ENCONTRADO,SIM,CENTRO,NAO,E-SUS,BRASIL,NAO
169,430010,AGUDO,1,SANTA MARIA - R01 R02,Feminino,60 a 69,TESTE RÁPIDO,21/09/2020,16/09/2020,NaN,...,NaN,30/09/2020,NAO INFORMADO,NAO ENCONTRADO,SIM,CENTRO,NAO,E-SUS,BRASIL,NAO
170,430010,AGUDO,1,SANTA MARIA - R01 R02,Feminino,80 e mais,TESTE RÁPIDO,28/09/2020,18/09/2020,NaN,...,NaN,02/10/2020,BRANCA,NAO ENCONTRADO,NAO,CENTRO,NAO,E-SUS,BRASIL,NAO


Atributos desconsiderados:
* COD_IBGE
* COD_REGIAO_COVID
* DATA_SINTOMAS
* DATA_EVOLUCAO
* DATA_INCLUSAO_OBITO
* DATA_EVOLUCAO_ESTIMADA
* ETNIA_INDIGENA
* FONTE_INFORMACAO
* PAIS_NASCIMENTO
* BAIRRO

#### 1) Remover colunas dos atributos desconsiderados <a name="remove_column"></a>

In [3]:
for e in cd.attr_removed:
    del df[e]

#### 2) Arrumando os dados - FAIXAETARIA, DATA_CONFIRMACAO, CONDICOES, BAIRRO <a name="data_fix"></a>

Faixa Etária
* < 1: Bebê
* (01,09): Criança
* (10, 19): Jovem
* (20, 59): Adulto
* (60, 79): Idoso 1
* (80 +): Idoso 2
       

In [4]:
for index, row in df.iterrows():
    if row['FAIXAETARIA'] == '<1':
        row['FAIXAETARIA'] = 'Bebê'
    elif row['FAIXAETARIA'] == '01 a 04' or row['FAIXAETARIA'] == '05 a 09':
        row['FAIXAETARIA'] = 'Criança'
    elif row['FAIXAETARIA'] == '10 a 14' or row['FAIXAETARIA'] == '15 a 19':
        row['FAIXAETARIA'] = 'Jovem'
    elif row['FAIXAETARIA'] == '20 a 29' or row['FAIXAETARIA'] == '30 a 39' or row['FAIXAETARIA'] == '40 a 49' or row['FAIXAETARIA'] == '50 a 59':
        row['FAIXAETARIA'] = 'Adulto'
    elif row['FAIXAETARIA'] == '60 a 69' or row['FAIXAETARIA'] == '70 a 79':
        row['FAIXAETARIA'] = 'Idoso_1'
    else:
        row['FAIXAETARIA'] = 'Idoso_2'

Data de Confirmação

In [5]:
for index, row in df.iterrows():
    mes = row['DATA_CONFIRMACAO'].split('/')
    if mes[1] == '02':
        row['DATA_CONFIRMACAO'] = 'Fevereiro'
    elif mes[1] == '03':
        row['DATA_CONFIRMACAO'] = 'Março'
    elif mes[1] == '04':
        row['DATA_CONFIRMACAO'] = 'Abril'
    elif mes[1] == '05':
        row['DATA_CONFIRMACAO'] = 'Maio'    
    elif mes[1] == '06':
        row['DATA_CONFIRMACAO'] = 'Junho'
    elif mes[1] == '07':
        row['DATA_CONFIRMACAO'] = 'Julho'
    elif mes[1] == '08':
        row['DATA_CONFIRMACAO'] = 'Agosto'
    elif mes[1] == '09':
        row['DATA_CONFIRMACAO'] = 'Setembro'
    else:
        row['DATA_CONFIRMACAO'] = 'Outubro'

Condições (Comorbidades)

In [6]:
for e in cd.comorbidades:
    df[e] = str(np.nan)

In [7]:
for index, row in df.iterrows():
    if 'asma' in str(row['CONDICOES']).lower():
        row['asma'] = 'SIM'
    else:
        row['asma'] = 'NAO'
    if 'respirató' in str(row['CONDICOES']).lower():
        row['d_respiratoria'] = 'SIM'
    else:
        row['d_respiratoria'] = 'NAO'
    if 'outro' in str(row['CONDICOES']).lower():
        row['outras_comorbidades'] = 'SIM'
    else:
        row['outras_comorbidades'] = 'NAO'
    if 'outro' in str(row['CONDICOES']).lower():
        row['outras_comorbidades'] = 'SIM'
    else:
        row['outras_comorbidades'] = 'NAO'
    if 'cromossômicas' in str(row['CONDICOES']).lower():
        row['d_cromossomica'] = 'SIM'
    else:
        row['d_cromossomica'] = 'NAO'
    if 'neuro' in str(row['CONDICOES']).lower():
        row['d_neurologica'] = 'SIM'
    else:
        row['d_neurologica'] = 'NAO'
    if 'diabe' in str(row['CONDICOES']).lower():
        row['diabetes'] = 'SIM'
    else:
        row['diabetes'] = 'NAO'
    if 'imuno' in str(row['CONDICOES']).lower():
        row['imunodeficiencia'] = 'SIM'
    else:
        row['imunodeficiencia'] = 'NAO'
    if 'rena' in str(row['CONDICOES']).lower():
        row['d_renal'] = 'SIM'
    else:
        row['d_renal'] = 'NAO'
    if 'hepát' in str(row['CONDICOES']).lower():
        row['d_hepatica'] = 'SIM'
    else:
        row['d_hepatica'] = 'NAO'
    if 'hemato' in str(row['CONDICOES']).lower():
        row['d_hematologica'] = 'SIM'
    else:
        row['d_hematologica'] = 'NAO'
    if 'card' in str(row['CONDICOES']).lower():
        row['d_cardiaca'] = 'SIM'
    else:
        row['d_cardiaca'] = 'NAO'
    if 'obes' in str(row['CONDICOES']).lower():
        row['obesidade'] = 'SIM'
    else:
        row['obesidade'] = 'NAO'
    if 'down' in str(row['CONDICOES']).lower():
        row['sindrome_down'] = 'SIM'
    else:
        row['sindrome_down'] = 'NAO'
    if 'puér' in str(row['CONDICOES']).lower():
        row['sindrome_down'] = 'SIM'
    else:
        row['puerpera'] = 'NAO'
del df['CONDICOES']

Bairro -- perguntar se devemos considerar

In [ ]:
# for index, row in df.iterrows():
#     if row['BAIRRO'] == 'INERIOR' or row['BAIRRO'] == 'INTERIOR ':
#         row['BAIRRO'] = 'INTERIOR'
#     if row['BAIRRO'] == 'VILA RECH' or row['BAIRRO'] == 'TIRADENTES' or \
#     row['BAIRRO'] == 'CAICARA' or str(row['BAIRRO']) == 'nan':
#         row['BAIRRO'] = 'OUTRO'

### Gerando a matrix binária <a name="binary_matrix"></a>

In [8]:
for e in cd.municipio:
    df[e] = df.apply(lambda row: 1 if row.MUNICIPIO == e else 0, axis=1)
    
for e in cd.regiao_covid:
    df[e] = df.apply(lambda row: 1 if row.REGIAO_COVID == e else 0, axis=1)
    
for e in cd.faixa_etaria:
    df[e] = df.apply(lambda row: 1 if row.FAIXAETARIA == e else 0, axis=1)
    
for e in cd.testagem: 
    df[e] = df.apply(lambda row: 1 if row.CRITERIO == e else 0, axis=1)

for e in cd.meses:
    df[e] = df.apply(lambda row: 1 if row.DATA_CONFIRMACAO == e else 0, axis=1)
    
for e in cd.evolucao:
    df[e] = df.apply(lambda row: 1 if row.EVOLUCAO == e else 0, axis=1)
    
for e in cd.etnia:
    df[e] = df.apply(lambda row: 1 if row.RACA_COR == e else 0, axis=1)

# for e in cd.bairro:
#     df[e] = df.apply(lambda row: 1 if row.BAIRRO == e else 0, axis=1)

In [9]:
df['SexF'] = df.apply(lambda row: 1 if row.SEXO == 'Feminino' else 0, axis=1)
df['SexM'] = df.apply(lambda row: 1 if row.SEXO == 'Masculino' else 0, axis=1)
df['Hosp_S'] = df.apply(lambda row: 1 if row.HOSPITALIZADO == 'SIM' else 0, axis=1)
df['Hosp_N'] = df.apply(lambda row: 1 if row.HOSPITALIZADO == 'NAO' else 0, axis=1)
df['Hosp_SRAG_S'] = df.apply(lambda row: 1 if row.HOSPITALIZACAO_SRAG == 'SIM' else 0, axis=1)
df['Hosp_SRAG_N'] = df.apply(lambda row: 1 if row.HOSPITALIZACAO_SRAG == 'NAO' else 0, axis=1)
df['Prof_Saude_S'] = df.apply(lambda row: 1 if row.PROFISSIONAL_SAUDE == 'SIM' else 0, axis=1)
df['Prof_Saude_N'] = df.apply(lambda row: 1 if row.PROFISSIONAL_SAUDE == 'SIM' else 0, axis=1)
df['Febre_S'] = df.apply(lambda row: 1 if row.FEBRE == 'SIM' else 0, axis=1)
df['Febre_N'] = df.apply(lambda row: 1 if row.FEBRE == 'NAO' else 0, axis=1)
df['Tosse_S'] = df.apply(lambda row: 1 if row.TOSSE == 'SIM' else 0, axis=1)
df['Tosse_N'] = df.apply(lambda row: 1 if row.TOSSE == 'NAO' else 0, axis=1)
df['Garganta_S'] = df.apply(lambda row: 1 if row.GARGANTA == 'SIM' else 0, axis=1)
df['Garganta_N'] = df.apply(lambda row: 1 if row.GARGANTA == 'NAO' else 0, axis=1)
df['Dispneia_S'] = df.apply(lambda row: 1 if row.DISPNEIA == 'SIM' else 0, axis=1)
df['Dispneia_N'] = df.apply(lambda row: 1 if row.DISPNEIA == 'NAO' else 0, axis=1)
df['Gestante_S'] = df.apply(lambda row: 1 if row.GESTANTE == 'SIM' else 0, axis=1)
df['Gestante_N'] = df.apply(lambda row: 1 if row.GESTANTE == 'NAO' else 0, axis=1)
df['Presidiario_S'] = df.apply(lambda row: 1 if row.PES_PRIV_LIBERDADE == 'SIM' else 0, axis=1)
df['Presidiario_N'] = df.apply(lambda row: 1 if row.PES_PRIV_LIBERDADE == 'NAO' else 0, axis=1)
df['Asma_S'] = df.apply(lambda row: 1 if row.asma == 'SIM' else 0, axis=1)
df['Asma_N'] = df.apply(lambda row: 1 if row.asma == 'NAO' else 0, axis=1)
df['DRespiratoria_S'] = df.apply(lambda row: 1 if row.d_respiratoria == 'SIM' else 0, axis=1)
df['DRespiratoria_N'] = df.apply(lambda row: 1 if row.d_respiratoria == 'NAO' else 0, axis=1)
df['DNeurologica_S'] = df.apply(lambda row: 1 if row.d_neurologica == 'SIM' else 0, axis=1)
df['DNeurologica_N'] = df.apply(lambda row: 1 if row.d_neurologica == 'NAO' else 0, axis=1)
df['Diabetes_S'] = df.apply(lambda row: 1 if row.diabetes == 'SIM' else 0, axis=1)
df['Diabetes_N'] = df.apply(lambda row: 1 if row.diabetes == 'NAO' else 0, axis=1)
df['Imunodef_S'] = df.apply(lambda row: 1 if row.imunodeficiencia == 'SIM' else 0, axis=1)
df['Imunodef_N'] = df.apply(lambda row: 1 if row.imunodeficiencia == 'NAO' else 0, axis=1)
df['DRenal_S'] = df.apply(lambda row: 1 if row.d_renal == 'SIM' else 0, axis=1)
df['DRenal_N'] = df.apply(lambda row: 1 if row.d_renal == 'NAO' else 0, axis=1)
df['DHepatica_S'] = df.apply(lambda row: 1 if row.d_hepatica == 'SIM' else 0, axis=1)
df['DHepatica_N'] = df.apply(lambda row: 1 if row.d_hepatica == 'NAO' else 0, axis=1)
df['DCardiaca_S'] = df.apply(lambda row: 1 if row.d_cardiaca == 'SIM' else 0, axis=1)
df['DCardiaca_N'] = df.apply(lambda row: 1 if row.d_cardiaca == 'NAO' else 0, axis=1)
df['Obesidade_S'] = df.apply(lambda row: 1 if row.obesidade == 'SIM' else 0, axis=1)
df['Obesidade_N'] = df.apply(lambda row: 1 if row.obesidade == 'NAO' else 0, axis=1)
df['Outras_Comorbidades_S'] = df.apply(lambda row: 1 if row.outras_comorbidades == 'SIM' else 0, axis=1)
df['Outras_Comorbidades_N'] = df.apply(lambda row: 1 if row.outras_comorbidades == 'NAO' else 0, axis=1)
df['DCromossomica_S'] = df.apply(lambda row: 1 if row.d_cromossomica == 'SIM' else 0, axis=1)
df['DCromossomica_N'] = df.apply(lambda row: 1 if row.d_cromossomica == 'NAO' else 0, axis=1)
df['DHematologica_S'] = df.apply(lambda row: 1 if row.d_hematologica == 'SIM' else 0, axis=1)
df['DHematologica_N'] = df.apply(lambda row: 1 if row.d_hematologica == 'NAO' else 0, axis=1)
df['Sindrome_down_S'] = df.apply(lambda row: 1 if row.sindrome_down == 'SIM' else 0, axis=1)
df['Sindrome_down_N'] = df.apply(lambda row: 1 if row.sindrome_down == 'NAO' else 0, axis=1)
df['Puerpera_S'] = df.apply(lambda row: 1 if row.puerpera == 'SIM' else 0, axis=1)
df['Puerpera_N'] = df.apply(lambda row: 1 if row.puerpera == 'NAO' else 0, axis=1)

In [10]:
for e in cd.old_columns:
    del df[e]
df.head()

,ACEGUÁ,ÁGUA SANTA,AGUDO,AJURICABA,ALECRIM,ALEGRETE,ALEGRIA,ALMIRANTE TAMANDARÉ DO SUL,ALPESTRE,ALTO ALEGRE,...,Outras_Comorbidades_S,Outras_Comorbidades_N,DCromossomica_S,DCromossomica_N,DHematologica_S,DHematologica_N,Sindrome_down_S,Sindrome_down_N,Puerpera_S,Puerpera_N
0,1,0,0,0,0,0,0,0,0,0,...,0,1,0,1,0,1,0,1,0,1
1,1,0,0,0,0,0,0,0,0,0,...,0,1,0,1,0,1,0,1,0,1
2,1,0,0,0,0,0,0,0,0,0,...,0,1,0,1,0,1,0,1,0,1
3,0,1,0,0,0,0,0,0,0,0,...,0,1,0,1,0,1,0,1,0,1
4,0,1,0,0,0,0,0,0,0,0,...,0,1,0,1,0,1,0,1,0,1


In [ ]:
# df.to_csv("binary_matrix.csv", header=None, index=False)

In [ ]:
nome_atributos = df.columns.values.tolist()
df_dict = {i : nome_atributos[i] for i in range(0, len(nome_atributos))}

### PaNDa<a name="panda"></a>

In [ ]:
for col,row in df.iteritems():
    df.loc[df[col] == 0, col] = 0
    df.loc[df[col] == 1, col] = int(df.columns.get_loc(col))

In [ ]:
panda = df.to_numpy()

In [ ]:
res = [[ele for ele in sub if ele != 0] for sub in panda]

In [ ]:
pd.DataFrame(res).to_csv("panda.csv", header=None, index=False, sep =" ")